In [ ]:
import cv2
import os
import yt_dlp
from datetime import datetime
from pathlib import Path
import shutil
import sys
from ultralytics import YOLO


def capture_youtube_to_folder(youtube_url, cam_id="Cam_1", project_name="CheckPPE", interval_sec=10):
    # 1. THIẾT LẬP ĐƯỜNG DẪN
    # Sử dụng Path để xử lý đường dẫn Windows có khoảng trắng và tiếng Việt tốt hơn
    output_dir = Path(r"D:\THS\06.1 Python\AI Camera\1_Check")
    
    if not output_dir.exists():
        output_dir.mkdir(parents=True, exist_ok=True)
        print(f"📂 Đã tạo thư mục: {output_dir}")

    # 2. LẤY LINK STREAM TỪ YOUTUBE
    ydl_opts = {'format': 'best[ext=mp4]', 'quiet': True}
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(youtube_url, download=False)
            video_url = info['url']
            print(f"✅ Đang kết nối video: {info['title']}")
    except Exception as e:
        print(f"❌ Lỗi kết nối YouTube: {e}")
        return

    # 3. XỬ LÝ VIDEO
    cap = cv2.VideoCapture(video_url)
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0: fps = 30
    
    frame_step = int(fps * interval_sec)
    current_frame = 0
    
    print(f"🚀 Bắt đầu cắt ảnh mỗi {interval_sec} giây...")
    print(f"📸 Quy tắc đặt tên: ID_Name_YYYYMMDD_HHMMSS.jpg")

    try:
        while True:
            cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
            ret, frame = cap.read()
            
            if not ret:
                break
            
            # LẤY THỜI GIAN HIỆN TẠI ĐỂ LÀM TÊN FILE
            # Định dạng: Ngày_Tháng_Năm_Gio_Phut_Giay
            now = datetime.now()
            time_str = now.strftime("%d_%m_%Y_%H_%M_%S")
            
            # Tên file theo rule: ID_Name_Thời gian
            file_name = f"{cam_id}_{project_name}_{time_str}.jpg"
            file_path = output_dir / file_name
            
            # Lưu ảnh
            cv2.imwrite(str(file_path), frame)
            print(f"   💾 Đã lưu: {file_name}")
            
            # Nhảy tới frame tiếp theo
            current_frame += frame_step
            
    except KeyboardInterrupt:
        print("\n⏹️ Đã dừng chương trình thủ công.")
    finally:
        cap.release()
        print("\n🎉 Hoàn tất xử lý.")

# ==============================================================================
# THỰC THI
# ==============================================================================
link_youtube = "https://www.youtube.com/watch?v=LCauiIO0Bt4"
#link_youtube = "https://www.youtube.com/watch?v=5BVLM3RZS2Y"


capture_youtube_to_folder(
    youtube_url = link_youtube,
    cam_id = "Cam_1",
    project_name = "PPE_Check", # Bạn có thể đổi tên dự án tại đây
    interval_sec = 10
)

# ==============================================================================
# 1. CẤU HÌNH & KHỞI TẠO
# ==============================================================================
def setup_system(base_path_str):
    """
    Thiết lập các đường dẫn và tạo thư mục nếu chưa tồn tại.
    Trả về dictionary chứa các đường dẫn.
    """
    base_dir = Path(base_path_str)
    
    paths = {
        "check":      base_dir / "1_Check",
        "helmet":     base_dir / "2_Detect_Helmet",
        "no_helmet":  base_dir / "3_Detect_No_Helmet",
        "no_detect":  base_dir / "5_No Detect",
        "archive":    base_dir / "9_Archieve"
    }

    # Tạo thư mục
    for p in paths.values():
        p.mkdir(parents=True, exist_ok=True)
        
    return base_dir, paths

# ==============================================================================
# 2. HÀM XỬ LÝ CHÍNH
# ==============================================================================
def run_helmet_detection(base_path_str):
    # --- A. Thiết lập ---
    BASE_DIR, PATHS = setup_system(base_path_str)
    
    # Từ khóa phân loại
    KEYWORDS_SAFE = ['helmet', 'hardhat', 'safety-cap']
    KEYWORDS_UNSAFE = ['no-helmet', 'head', 'face', 'person', 'no_helmet']

    # --- B. Load Model ---
    model_path = BASE_DIR / "best.pt"
    if not model_path.exists():
        print(f"❌ Lỗi: Không tìm thấy model tại {model_path}")
        return # Dừng hàm nếu không có model

    print(f"⏳ Đang tải model từ {model_path}...")
    try:
        model = YOLO(str(model_path))
    except Exception as e:
        print(f"❌ Lỗi khi tải model: {e}")
        return

    # --- C. Tìm ảnh đầu vào ---
    image_files = list(PATHS["check"].glob('*.jpg')) + list(PATHS["check"].glob('*.png'))

    if not image_files:
        print(f"⚠️ Thư mục {PATHS['check']} đang trống. Không có ảnh để xử lý.")
        return

    print(f"🚀 Đang xử lý {len(image_files)} ảnh (Chế độ vẽ khung riêng biệt)...")

    # --- D. Chạy nhận diện (Batch Inference) ---
    # Sử dụng iou=0.3 để khử khung chồng lấn, conf=0.1 để bắt lỗi nhạy
    results = model.predict(source=image_files, conf=0.1, iou=0.3, verbose=False)

    # --- E. Xử lý từng kết quả ---
    for result in results:
        origin_file = Path(result.path)
        filename = origin_file.name
        
        # Đọc ảnh gốc
        img = cv2.imread(str(origin_file))
        if img is None:
            print(f"❌ Lỗi đọc file: {filename}")
            continue

        target_folder = PATHS["no_detect"]
        has_violation = False
        has_safe = False
        
        boxes = result.boxes
        if len(boxes) > 0:
            for box in boxes:
                cls_name = model.names[int(box.cls[0])].lower().replace("_", "-")
                conf_score = float(box.conf[0])
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                
                # --- LOGIC VẼ KHUNG ---
                
                # 1. Kiểm tra nhóm VI PHẠM (Ưu tiên vẽ Đỏ)
                is_unsafe = any(kw in cls_name for kw in KEYWORDS_UNSAFE)
                if is_unsafe and conf_score >= 0.1:
                    has_violation = True
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
                    cv2.putText(img, f"NO-HELMET {conf_score:.2f}", (x1, y1 - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                
                # 2. Kiểm tra nhóm AN TOÀN (Vẽ Xanh)
                is_safe = any(kw in cls_name for kw in KEYWORDS_SAFE)
                if is_safe and conf_score >= 0.4:
                    # Chỉ vẽ xanh nếu chưa bị đánh dấu là đỏ (để tránh rối mắt)
                    if not has_violation:
                        has_safe = True
                        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(img, f"Helmet {conf_score:.2f}", (x1, y1 - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                    else:
                        # Nếu ảnh đã có vi phạm, vẫn đánh dấu logic safe=True để biết, 
                        # nhưng không vẽ lên hình để tập trung vào lỗi.
                        has_safe = True

            # --- Quyết định thư mục lưu ---
            if has_violation:
                target_folder = PATHS["no_helmet"]
            elif has_safe:
                target_folder = PATHS["helmet"]

        # --- F. Lưu và Di chuyển ---
        # 1. Lưu ảnh kết quả
        output_path = target_folder / filename
        cv2.imwrite(str(output_path), img)
        
        # 2. Di chuyển ảnh gốc sang Archive
        archive_path = PATHS["archive"] / filename
        shutil.move(str(origin_file), str(archive_path))

        status = "❌ VI PHẠM" if has_violation else ("✅ AN TOÀN" if has_safe else "⚪ KHÔNG DETECT")
        print(f"{status} | {filename} -> {target_folder.name}")

    print("\n🎉 Hoàn tất! Đã xử lý xong toàn bộ ảnh.")

# ==============================================================================
# 3. CHẠY CHƯƠNG TRÌNH
# ==============================================================================
if __name__ == "__main__":
    # Bạn chỉ cần đổi đường dẫn ở đây
    working_path = r"D:\THS\06.1 Python\AI Camera"
    
    run_helmet_detection(working_path)


✅ Đang kết nối video: Saudi Arab Riyadh Randa company
🚀 Bắt đầu cắt ảnh mỗi 10 giây...
📸 Quy tắc đặt tên: ID_Name_YYYYMMDD_HHMMSS.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_15_57_09.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_15_57_42.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_15_58_08.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_15_59_03.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_16_00_25.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_16_01_17.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_16_02_15.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_16_02_29.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_16_03_32.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_16_04_27.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_16_05_14.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_16_05_52.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_16_05_59.jpg
   💾 Đã lưu: Cam_1_PPE_Check_30_12_2025_16_06_08.jpg

🎉 Hoàn tất xử lý.
⏳ Đang tải model từ D:\THS\06.1 Python\AI Camera\best.pt...
🚀 Đang xử lý 14 ảnh (Chế độ vẽ khung riêng bi